In [ ]:
"""
Created on Mon Apr 08 16:22 2024

This script is to make a timeseries of runs that have crossed the viability limit

@author: Clara Burgard
"""

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import cm
import matplotlib as mpl
import cmocean
import glob
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import os
import multimelt.useful_functions as uf

In [ ]:
sns.set_context('paper')

In [ ]:
%matplotlib qt5

In [ ]:
# make the domain a little smaller to make the computation even more efficient - file isf has already been made smaller at its creation
map_lim = [-3000000,3000000]

READ IN DATA

In [ ]:
home_path = '/bettik/burgardc/'
plot_path = '/bettik/burgardc/PLOTS/summer_paper_plots/'
outputpath_GL = '/bettik/burgardc/DATA/SUMMER_PAPER/processed/GL_FLUX/'
inputpath_weights = '/bettik/burgardc/DATA/SUMMER_PAPER/processed/ANALYSIS/'
inputpath_atmo = '/bettik/burgardc/DATA/SUMMER_PAPER/raw/TS_SMB_DATA/out/'
inputpath_data='/bettik/burgardc/DATA/SUMMER_PAPER/interim/'


In [ ]:
inputpath_mask = home_path+'/DATA/SUMMER_PAPER/interim/ANTARCTICA_IS_MASKS/BedMachine_4km/'
file_isf_orig = xr.open_dataset(inputpath_mask+'BedMachinev2_4km_isf_masks_and_info_and_distance_oneFRIS.nc')
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
rignot_isf = file_isf_nonnan.Nisf.where(np.isfinite(file_isf_nonnan['isf_area_rignot']), drop=True)
file_isf = file_isf_nonnan.sel(Nisf=rignot_isf)

In [ ]:
weight_file = xr.open_dataset(inputpath_weights + 'bayesian_weights_davison_varying_combined_withoutGISS.nc')
weight_2300_file = xr.open_dataset(inputpath_weights + 'bayesian_weights_davison_varying_combined_2300_withoutGISS.nc')
file_viability_info = xr.open_dataset(inputpath_weights + 'all_fluxes_br_withoutGISS.nc')

In [ ]:
ano_choice = xr.open_dataset(inputpath_weights + 'ano_choice_NEMOorISMIP_withoutGISS.nc')['ano_choice']

In [ ]:
file_TS_scen_list = []
for scenario in ['ssp126','ssp245','ssp585']:
    print(scenario)
    
    file_TS_model_list = []
    for mod in weight_file.model.values:
        print(mod)

        if mod in ['CNRM-CM6-1','CNRM-ESM2-1']:
            to2300 = False
        elif mod in ['GISS-E2-1-H']:
            to2300 = True
        elif mod in ['ACCESS-CM2','ACCESS-ESM1-5','CESM2-WACCM','CanESM5','IPSL-CM6A-LR','MRI-ESM2-0']:
            to2300 = True
        elif mod in ['MPI-ESM1-2-HR','GFDL-CM4','GFDL-ESM4']:
            to2300 = False
        elif mod == 'UKESM1-0-LL':
            to2300 = True     
        elif mod == 'CESM2':
            to2300 = False        

        if scenario == 'historical':
            yystart = 1850 #1980 #1850
            yyend = 2014
        elif scenario == 'ssp245':
            yystart = 2015
            yyend = 2100  
        else:
            if to2300:
                yystart = 2015
                yyend = 2300
            else:
                yystart = 2015
                yyend = 2100 

        inputpath_profiles = home_path+'/DATA/SUMMER_PAPER/interim/T_S_PROF/CMIP/'+mod+'/'

        T_Clim = xr.open_dataset(inputpath_profiles + 'T_mean_prof_50km_contshelf_'+mod+'_clim.nc')
        S_Clim = xr.open_dataset(inputpath_profiles + 'S_mean_prof_50km_contshelf_'+mod+'_clim.nc')
        TS_Clim = xr.merge([T_Clim.rename({'thetao':'theta_ocean'}),S_Clim.rename({'so':'salinity_ocean'})])
        
        if scenario == 'ssp126' and mod == 'GFDL-CM4':
            print('beuh')
        else:
        
            file_TS_list = []
            for tt in tqdm(range(yystart,yyend+1)):
                file_T_orig = xr.open_dataset(inputpath_profiles+'T_mean_prof_50km_contshelf_'+mod+'_'+scenario+'_'+str(tt)+'.nc')
                file_S_orig = xr.open_dataset(inputpath_profiles+'S_mean_prof_50km_contshelf_'+mod+'_'+scenario+'_'+str(tt)+'.nc')
                file_TS_orig = xr.merge([file_T_orig.rename({'thetao':'theta_ocean'}), file_S_orig.rename({'so':'salinity_ocean'})]).sel(Nisf=rignot_isf).assign_coords({'time': tt})
                file_TS_list.append(file_TS_orig)
            file_TS_0 = xr.concat(file_TS_list, dim='time') 
            file_TS_model_list.append(file_TS_0.assign_coords({'model': mod}))
        file_TS1 = xr.concat(file_TS_model_list, dim='model')
    file_TS_scen_list.append(file_TS1.assign_coords({'scenario': scenario}))
file_TS2 = xr.concat(file_TS_scen_list, dim='scenario')

In [ ]:
file_TS2.to_netcdf(home_path+'/DATA/SUMMER_PAPER/interim/T_S_PROF/CMIP/T_S_prof_allfuturescen_allmod.nc')

In [ ]:
file_TS2 = xr.open_dataset(home_path+'/DATA/SUMMER_PAPER/interim/T_S_PROF/CMIP/T_S_prof_allfuturescen_allmod.nc')

In [ ]:
Tmax = file_TS2['theta_ocean'].max('z')

In [ ]:
Tmean = file_TS2['theta_ocean'].mean('z')

In [ ]:
scen = 'ssp126'

f = plt.figure()
f.set_size_inches(8.25*3, 8.25*3)

ax={}

i = 0

for kisf in tqdm(file_isf.Nisf.sel(Nisf=sorted_isf_rignot)):

    ax[i] = f.add_subplot(8,8,i+1)
                    
    ax[i].plot(np.arange(2015,2300),Tmax.sel(model='GISS-E2-1-H',scenario=scen, time=range(2015,2300), Nisf=kisf), color='orange')

            
    i = i+1
    
sns.despine()
plt.tight_layout()
f.savefig(plot_path + 'Tmean_GISS_'+scen+'.png',dpi=300)

In [ ]:
file_TS_scen_list = []
for scenario in ['historical','ssp126','ssp245','ssp585']:
    print(scenario)
    
    file_TS_model_list = []
    for mod in tqdm(weight_file.model.values):

        if mod in ['CNRM-CM6-1','CNRM-ESM2-1']:
            to2300 = False
        elif mod in ['GISS-E2-1-H']:
            to2300 = True
        elif mod in ['ACCESS-CM2','ACCESS-ESM1-5','CESM2-WACCM','CanESM5','IPSL-CM6A-LR','MRI-ESM2-0']:
            to2300 = True
        elif mod in ['MPI-ESM1-2-HR','GFDL-CM4','GFDL-ESM4']:
            to2300 = False
        elif mod == 'UKESM1-0-LL':
            to2300 = True     
        elif mod == 'CESM2':
            to2300 = False        

        if scenario == 'historical':
            yystart = 1850 #1980 #1850
            yyend = 2014
        elif scenario == 'ssp245':
            yystart = 2015
            yyend = 2100  
        else:
            if to2300:
                yystart = 2015
                yyend = 2300
            else:
                yystart = 2015
                yyend = 2100 

        outputpath_mean = home_path+'/DATA/SUMMER_PAPER/interim/T_S_mean/'+mod+'/'
        
        if scenario == 'ssp126' and mod == 'GFDL-CM4':
            print('beuh')
        else:
            mean_prop = xr.open_dataset(outputpath_mean + 'T_S_TF_isfmean_'+scenario+'.nc')
            file_TS_model_list.append(mean_prop.assign_coords({'model': mod}))
        file_TS1 = xr.concat(file_TS_model_list, dim='model')
    file_TS_scen_list.append(file_TS1.assign_coords({'scenario': scenario}))
file_TS3 = xr.concat(file_TS_scen_list, dim='scenario')

In [ ]:
Tmax_quantiles = weighted_Tmax.quantile([0.33,0.5,0.66],dim=['model','param','m'])
Tmean_quantiles = weighted_Tmean.quantile([0.33,0.5,0.66],dim=['model','param','m'])

PREPARE THE DATA

In [ ]:
sorted_isf_rignot = [11,69,43,28,12,57,
                     70,44,29,13,58,71,45,30,14,
                     59,72,46,
                     31,
                     15,61,73,47,32,16,48,33,17,62,49,34,18,63,74,
                     50,35,19,64,
                     10,
                     36,20,65,51,37,
                     22,38,52,23,66,53,39,24,
                     67,40,54,75,25,41,
                     26,42,55,68,60,27]

In [ ]:
sens_weights = xr.DataArray(data=np.array([0.11,
                                           0.24,
                                           0.03,
                                           0.10,
                                           0.10,
                                           0.10,
                                           0.10,
                                           0.24,
                                           0.47,
                                           0.41,
                                           0.12,
                                           0.43,
                                           0.39,
                                           0.05]), dims=['model']).assign_coords({'model': 
                                                                                  ['ACCESS-CM2','ACCESS-ESM1-5','CanESM5',
                                                                                   'CESM2','CESM2-WACCM','CNRM-CM6-1','CNRM-ESM2-1',
                                                                                   'GFDL-CM4','GFDL-ESM4','GISS-E2-1-H', 'IPSL-CM6A-LR',
                                                                                   'MPI-ESM1-2-HR','MRI-ESM2-0','UKESM1-0-LL']})
sens_weights = sens_weights.drop_sel(model='GISS-E2-1-H')

In [ ]:
model_2300 = ['ACCESS-CM2','ACCESS-ESM1-5','CanESM5','CESM2-WACCM', 'IPSL-CM6A-LR','MRI-ESM2-0','UKESM1-0-LL'] #'GISS-E2-1-H',

In [ ]:
bay_weights = weight_file['bay_weights']
bay_weights_2300 = weight_2300_file['bay_weights']

In [ ]:
mass_balance_weighted_yy = file_viability_info['MASS_BALANCE'].sel(time=range(1850,2101)).weighted(bay_weights * sens_weights)
mass_balance_weighted_yy_2300 = file_viability_info['MASS_BALANCE'].sel(time=range(1850,2300)).sel(model=model_2300).weighted(bay_weights_2300 * sens_weights.sel(model=model_2300))

In [ ]:
weighted_quantiles = mass_balance_weighted_yy.quantile([0,0.1,0.33,0.66,0.5,0.9,1],dim=['model','param','m'])
weighted_quantiles_2300 = mass_balance_weighted_yy_2300.quantile([0,0.1,0.33,0.66,0.5,0.9,1],dim=['model','param','m'])

In [ ]:
unviable_times = weighted_quantiles > 0
unviable_times_2300 = weighted_quantiles_2300 > 0


In [ ]:
viability_diff = unviable_times.sel(time=range(2100,1849,-1)).cumsum('time').diff('time')
limit_max = viability_diff.time.where(viability_diff == 0).max('time') + 1
limit_max = limit_max.where(limit_max<2099)
limit_max = limit_max.where(~(np.isnan(limit_max) & unviable_times.sel(time=1850)), 1850)

limit_max_full = limit_max.where(np.isfinite(limit_max), 2301)

count_unviable_isf = (unviable_times.time >= limit_max_full).sum('Nisf')

In [ ]:
viability_diff_2300 = unviable_times_2300.sel(time=range(2298,1849,-1)).cumsum('time').diff('time')
limit_max_2300 = viability_diff_2300.time.where(viability_diff_2300 == 0).max('time') + 1
limit_max_2300 = limit_max_2300.where(limit_max_2300<2299)
limit_max_2300 = limit_max_2300.where(~(np.isnan(limit_max_2300) & unviable_times_2300.sel(time=1850)), 1850)

limit_max_full_2300 = limit_max_2300.where(np.isfinite(limit_max_2300), 2305)

count_unviable_isf_2300 = (unviable_times_2300.time >= limit_max_full_2300).sum('Nisf')

In [ ]:
mean_prop_weighted = file_TS3.weighted(bay_weights * sens_weights)
mean_prop_weighted_2300 = file_TS3.sel(model=model_2300).weighted(bay_weights_2300 * sens_weights.sel(model=model_2300))

In [ ]:
weighted_Tmax = Tmax.sel(model=model_2300).weighted(bay_weights_2300 * sens_weights.sel(model=model_2300))
weighted_Tmean = Tmean.sel(model=model_2300).weighted(bay_weights_2300 * sens_weights.sel(model=model_2300))

In [ ]:
Tmax_quantiles = weighted_Tmax.quantile([0.33,0.5,0.66],dim=['model','param','m'])
Tmean_quantiles = weighted_Tmean.quantile([0.33,0.5,0.66],dim=['model','param','m'])

In [ ]:
f = plt.figure()
f.set_size_inches(8.25*3, 8.25*3)

ax={}

anomaly_mb = file_viability_info['MASS_BALANCE'].sel(time=range(2015,2101)) - file_viability_info['MASS_BALANCE'].sel(time=range(2015,2101)).mean('time')

i = 0

nonviable_ano = anomaly_mb.where(file_viability_info['MASS_BALANCE'] > 0)

for kisf in tqdm(file_isf.Nisf.sel(Nisf=sorted_isf_rignot[::-1])):

    ax[i] = f.add_subplot(8,8,i+1)
        
    for scen in ['ssp126','ssp245','ssp585']:

        if scen == 'ssp126':
            ccol = 'gold'
        elif scen == 'ssp245':
            ccol = 'darkorange'
        elif scen == 'ssp585':
            ccol = 'red'
    
                    
        ax[i].scatter(Tmax.where(nonviable_ano).sel(scenario=scen,Nisf=kisf),anomaly_mb.where(nonviable_ano).sel(scenario=scen,Nisf=kisf), s=5, color=ccol, alpha=0.05)
        ax[i].set_title(file_isf['isf_name'].sel(Nisf=kisf).values)
            
    i = i+1
    
sns.despine()
plt.grid(linestyle='--')
plt.tight_layout()
f.savefig(plot_path + 'relationship_with_Tocean_atnonviability.png',dpi=300)

In [ ]:
weighted_prop_2300 = file_TS3.weighted(bay_weights_2300 * sens_weights.sel(model=model_2300)).quantile([0.33,0.5,0.66],dim=['model','param','m'])

In [ ]:
scen = 'ssp585'
var = 'TF_mean'

f = plt.figure()
f.set_size_inches(8.25*1.5, 8.25*1.5)

ax={}

i = 0

for kisf in tqdm(file_isf.Nisf.sel(Nisf=sorted_isf_rignot)):
    
    if i > 0:
        ax[i] = f.add_subplot(8,8,i+1, sharex=ax[0])
    else:
        ax[i] = f.add_subplot(8,8,i+1)

    
                    
    ax[i].plot(np.arange(1850,2015),weighted_prop_2300[var].sel(quantile=0.5, scenario=scen, time=range(1850,2015), Nisf=kisf), color='orange')

    ax[i].plot(np.arange(2015,2300),weighted_prop_2300[var].sel(quantile=0.5, scenario=scen, time=range(2015,2300), Nisf=kisf), color='orange')
    
    
    ax[i].set_title(file_isf['isf_name'].sel(Nisf=kisf).values)
    ax[i].set_xticks([1900,2000,2100,2200,2300])
    if i < 56:
        ax[i].tick_params(labelbottom=False)
    else:
        ax[i].set_xticklabels([1900, 2000, 2100, 2200, 2300], rotation=45)
    
            
    i = i+1
    
sns.despine()
plt.tight_layout()

In [ ]:
non_viable_ensmembers_2300 = file_viability_info['MASS_BALANCE'].sel(model=model_2300) > 0

non_via_diff_2300 = non_viable_ensmembers_2300.sel(time=range(2299,1849,-1)).cumsum('time').diff('time')
limit_via_max_2300 = non_via_diff_2300.time.where(non_via_diff_2300 == 0).max('time') + 1
limit_via_max_2300 = limit_via_max_2300.where(limit_via_max_2300<2299)
limit_via_max_2300 = limit_via_max_2300.where(~(np.isnan(limit_via_max_2300) & non_viable_ensmembers_2300.sel(time=1850)), 1850)

In [ ]:
time_br_2300, _ = xr.broadcast(file_viability_info.time, file_viability_info['MASS_BALANCE'].sel(model=model_2300))
file_TS3_br, _ = xr.broadcast(file_TS3, file_viability_info['MASS_BALANCE'].sel(model=model_2300))
non_viable_time_2300 = limit_via_max_2300
nonvia_prop_2300 = file_TS3_br.where(time_br_2300 == non_viable_time_2300, drop=True).max('time')


In [ ]:
nonvia_prop_2300_w = nonvia_prop_2300#.weighted(bay_weights_2300 * sens_weights.sel(model=model_2300)).mean(['param','m'])

In [ ]:
f = plt.figure()
f.set_size_inches(8.25, 8.25)

date_of_int = limit_max_full_2300.sel(quantile=0.33,scenario='ssp585') 
#prop_to_plot = mean_prop_weighted.mean(['m','param']).sel(scenario='ssp585')
#anom_TF = prop_to_plot['TF_mean'].sel(scenario='ssp585') - prop_to_plot['TF_mean'].sel(scenario='historical',time=range(1850,1901)).mean('time')

#plt.scatter(prop_to_plot['T_mean'].sel(time=date_of_int-1),prop_to_plot['T_mean'].sel(time=date_of_int))
#plt.scatter(anom_TF.sel(time=date_of_int-1),anom_TF.sel(time=date_of_int))
for model in nonvia_prop_2300_w.model:
    for m in nonvia_prop_2300_w.m:
        for param in bay_weights_2300.param:
            plt.scatter(nonvia_prop_2300_w['T_mean'].sel(m=m,model=model,param=param),nonvia_prop_2300_w['TF_mean'].sel(m=m,model=model,param=param), alpha=(bay_weights_2300 * sens_weights.sel(model=model_2300)).sel(m=m,model=model,param=param)*2)

sns.despine()
plt.grid(linestyle='--')
plt.tight_layout()


In [ ]:
param

In [ ]:
f = plt.figure()
f.set_size_inches(8.25, 8.25)

date_of_int = limit_max_full_2300.sel(quantile=0.33,scenario='ssp585') 
prop_to_plot = mean_prop_weighted_2300.mean(['m','param','model'])
nb_of_int = count_unviable_isf_2300.sel(quantile=0.33,scenario='ssp585')
anom_TF = prop_to_plot['TF_mean'].sel(scenario='ssp585') - prop_to_plot['TF_mean'].sel(scenario='historical',time=range(1850,1901)).mean('time')
plt.scatter((anom_TF >= 0.5).sum('Nisf').sel(time=nb_of_int.time),nb_of_int)
    
sns.despine()
plt.xlim(0,64)
plt.ylim(0,64)
plt.plot(range(64),range(64))
plt.grid(linestyle='--')
#plt.tight_layout()
plt.xlabel('Thermal forcing anomaly >= 0.5°C at non-viability')
plt.ylabel('Number of non-viable ice shelves')



In [ ]:
prop_to_plot

In [ ]:
scen = 'ssp585'

f = plt.figure()
f.set_size_inches(8.25*3, 8.25*3)

ax={}

i = 0


for kisf in tqdm(file_isf.Nisf.sel(Nisf=sorted_isf_rignot)):
    
    ax[i] = f.add_subplot(8,8,i+1)

    #for mmod in model_2300:


     #   ax[i].plot(np.arange(2015,2300),file_TS3['TF_mean'].sel(model=mmod,scenario='ssp585',Nisf=kisf,time=range(2015,2300)))
    ax[i].plot(np.arange(2015,2300),prop_to_plot['TF_mean'].sel(scenario='ssp585',Nisf=kisf,time=range(2015,2300)))

    ax[i].set_title(file_isf['isf_name'].sel(Nisf=kisf).values)
    ax[i].axvline(x=2100)

    i = i+1

sns.despine()
plt.tight_layout()
plt.legend()


Look at BMB in this relationship as well

In [ ]:
file_viability_info = xr.open_dataset(inputpath_weights + 'all_fluxes_br_withoutGISS.nc')

In [ ]:
BMB_weighted_mean = file_viability_info['BMB'].sel(model=model_2300).weighted(bay_weights_2300 * sens_weights.sel(model=model_2300)).mean(['model','param','m'])
GLflux_weighted_mean = file_viability_info['ABUMIP'].sel(model=model_2300).weighted(bay_weights_2300 * sens_weights.sel(model=model_2300)).mean(['model','param','m']).sel(scenario='ssp585',time=2020)

In [ ]:
smoothed_curve = BMB_weighted_mean.rolling(time=60,center=True).mean()
slope_before = (smoothed_curve.sel(scenario='ssp585') - smoothed_curve.shift(time=30).sel(scenario='ssp585'))
slope_after = (smoothed_curve.shift(time=-30).sel(scenario='ssp585') - smoothed_curve.sel(scenario='ssp585'))
slope_diff = slope_after - slope_before
slope_diff2 = smoothed_curve.shift(time=-30).sel(scenario='ssp585') - smoothed_curve.shift(time=30).sel(scenario='ssp585')
max_slope_time = slope_diff.time.isel(time=slope_diff.argmax('time'))

In [ ]:
smoothed_slope_diff2 = slope_diff2.rolling(time=30,center=True).mean()
slope_slope_diff2 = smoothed_slope_diff2.shift(time=-20) - smoothed_slope_diff2.shift(time=20)
max_slope2_time = slope_slope_diff2.argmax('time')

In [ ]:
date_of_int = limit_max_full_2300.sel(quantile=0.33,scenario='ssp585') 

In [ ]:
time_scale = GLflux_weighted_mean/slope_after.sel(time=max_slope_time)

In [ ]:
smoothed_curve = file_viability_info['BMB'].sel(model=model_2300).rolling(time=60,center=True).mean('time')
slope_before = (smoothed_curve.sel(scenario='ssp585') - smoothed_curve.shift(time=60).sel(scenario='ssp585'))
slope_after = (smoothed_curve.shift(time=-60).sel(scenario='ssp585') - smoothed_curve.sel(scenario='ssp585'))
slope_across = (smoothed_curve.shift(time=-60).sel(scenario='ssp585') - smoothed_curve.shift(time=60).sel(scenario='ssp585'))
slope_diff2 = slope_across.shift(time=-1) - slope_across.shift(time=1)
slope_diff = slope_after - slope_before 

In [ ]:
max_slope_time = slope_diff.time.isel(time=slope_diff.argmax('time'))
max_slope2_time = slope_diff2.time.isel(time=slope_diff2.argmax('time')) 

In [ ]:
# looks like max_slope2_time is good
mm = 2
mparam = 6
mmod = 3
kisf = 10
smoothed_curve.isel(m=mm,param=mparam,model=mmod,Nisf=kisf).sel(scenario='ssp585').plot()
plt.axvline(x=max_slope2_time.isel(m=mm,param=mparam,model=mmod,Nisf=kisf))
plt.axvline(x=max_slope_time.isel(m=mm,param=mparam,model=mmod,Nisf=kisf),linestyle='--')

In [ ]:
slopemom_prop_2300 = file_TS3_br.where(time_br_2300 == max_slope2_time, drop=True).max('time')


In [ ]:
slopemom_prop_2300['T_mean'].sel(m=m,model=model,param=param,scenario='ssp585',Nisf=kisf)

In [ ]:
max_slope2_time.sel(Nisf=kisf)

In [ ]:
scen = 'ssp585'

f = plt.figure()
f.set_size_inches(8.25*3, 8.25*3)

ax={}

i = 0


for kisf in tqdm(file_isf.Nisf.sel(Nisf=sorted_isf_rignot)):
    
    ax[i] = f.add_subplot(8,8,i+1)

    #for mmod in model_2300:
    for model in nonvia_prop_2300_w.model:
        for m in nonvia_prop_2300_w.m:
            for param in bay_weights_2300.param:

             #   ax[i].plot(np.arange(2015,2300),file_TS3['TF_mean'].sel(model=mmod,scenario='ssp585',Nisf=kisf,time=range(2015,2300)))
                ax[i].scatter(max_slope2_time.sel(m=m,model=model,param=param,Nisf=kisf),slopemom_prop_2300['TF_mean'].sel(m=m,model=model,param=param,scenario='ssp585',Nisf=kisf), c='deepskyblue', alpha=(bay_weights_2300 * sens_weights.sel(model=model_2300)).sel(m=m,model=model,param=param)*100)

    ax[i].set_title(file_isf['isf_name'].sel(Nisf=kisf).values)
    ax[i].axvline(x=2100)
    ax[i].set_ylim(0,4)

    i = i+1

sns.despine()
plt.tight_layout()
plt.legend()


In [ ]:
f = plt.figure()
f.set_size_inches(8.25, 8.25)


#plt.scatter(prop_to_plot['T_mean'].sel(time=date_of_int-1),prop_to_plot['T_mean'].sel(time=date_of_int))
#plt.scatter(anom_TF.sel(time=date_of_int-1),anom_TF.sel(time=date_of_int))
for model in nonvia_prop_2300_w.model:
    for m in nonvia_prop_2300_w.m:
        for param in bay_weights_2300.param:
            plt.scatter(slopemom_prop_2300['T_mean'].sel(m=m,model=model,param=param),slopemom_prop_2300['TF_mean'].sel(m=m,model=model,param=param), alpha=(bay_weights_2300 * sens_weights.sel(model=model_2300)).sel(m=m,model=model,param=param)*2)

sns.despine()
plt.grid(linestyle='--')
plt.tight_layout()


In [ ]:
max_slope_time.quantile([0.33,0.5,0.66])

In [ ]:
max_slope_time.std()

In [ ]:
scen = 'ssp585'

f = plt.figure()
f.set_size_inches(8.25*3, 8.25*3)

ax={}

i = 0


for kisf in tqdm(file_isf.Nisf.sel(Nisf=sorted_isf_rignot)):
    
    ax[i] = f.add_subplot(8,8,i+1)

    #ax[i].plot(np.arange(1850,2300),BMB_weighted_mean.sel(scenario='ssp585',Nisf=kisf))
    #ax[i].plot(np.arange(1850,2300),smoothed_curve.sel(scenario='ssp585',Nisf=kisf))
    ax[i].plot(np.arange(1850,2300),prop_to_plot['TF_mean'].sel(scenario='ssp585',Nisf=kisf,time=np.arange(1850,2300)))
    
    #ax[i].plot(np.arange(1850,2300),slope_diff.sel(Nisf=kisf))
    #ax[i].plot(np.arange(1850,2300),slope_slope_diff2.sel(Nisf=kisf))
    
    ax[i].text(1850,0,str(np.round(time_scale.sel(Nisf=kisf).values,0)))


    ax[i].set_title(file_isf['isf_name'].sel(Nisf=kisf).values)
    ax[i].axvline(x=max_slope_time.sel(Nisf=kisf), c='grey')
    ax[i].axvline(x=date_of_int.sel(Nisf=kisf), c='black')


    i = i+1

sns.despine()
plt.tight_layout()
plt.legend()


In [ ]:
TF_diff = prop_to_plot['TF_mean'].sel(scenario='ssp585',time=max_slope_time+30) - prop_to_plot['TF_mean'].sel(scenario='ssp585',time=max_slope_time-30)
plt.figure()
plt.scatter(TF_diff,slope_diff.sel(time=max_slope_time)/file_isf['isf_area_here'])

In [ ]:
plt.figure()
plt.scatter(BMB_weighted_mean.sel(scenario='ssp585',time=range(1850,1901)).mean('time')/file_isf['isf_area_here'], GLflux_weighted_mean/slope_after.sel(time=max_slope_time))
plt.xlabel('Mean historical melt [Gt/yr/km2]')
plt.ylabel('ABUMIPflux / slope in first 30 years start of large increase in basal melt [yr]')
sns.despine()

In [ ]:
plt.figure()
plt.scatter(prop_to_plot['TF_mean'].sel(scenario='historical',time=range(1850,1901)).mean('time'), GLflux_weighted_mean/slope_after.sel(time=max_slope_time))
plt.xlabel('Mean historical thermal forcing [°C]')
plt.ylabel('ABUMIPflux / slope in first 30 years start of large increase in basal melt [yr]')
sns.despine()

In [ ]:
plt.figure()
plt.scatter(BMB_weighted_mean.sel(scenario='ssp585',time=range(1850,1901)).mean('time')/file_isf['isf_area_here'], slope_after.sel(time=max_slope_time)/file_isf['isf_area_here'])
sns.despine()
plt.xlabel('Mean historical melt [Gt/yr/km2]')
plt.ylabel('Slope in first 30 years start of large increase in basal melt [Gt/yr/yr/km2]')

In [ ]:
nisf_of_int = file_isf.Nisf.sel(Nisf=[10,11,44])

plt.figure()
plt.scatter(prop_to_plot['TF_mean'].sel(scenario='ssp585',time=np.arange(1850,2300),Nisf=nisf_of_int),BMB_weighted_mean.sel(scenario='ssp585',Nisf=nisf_of_int), s=20, alpha=0.1)

In [ ]:
nisf_of_int = file_isf.Nisf.sel(Nisf=[66,23,48])
plt.figure()
plt.scatter(xr.broadcast(nisf_of_int,prop_to_plot.time)[0],prop_to_plot['TF_mean'].sel(scenario='ssp585',Nisf=nisf_of_int))
plt.scatter(nisf_of_int,prop_to_plot['TF_mean'].sel(scenario='ssp585',time=max_slope_time.sel(Nisf=nisf_of_int),Nisf=nisf_of_int))

In [ ]:
nisf_of_int = file_isf.Nisf.sel(Nisf=[10,11,44])
plt.figure()
plt.scatter(xr.broadcast(nisf_of_int,prop_to_plot.time)[0],prop_to_plot['TF_mean'].sel(scenario='ssp585',Nisf=nisf_of_int))
plt.scatter(nisf_of_int,prop_to_plot['TF_mean'].sel(scenario='ssp585',time=max_slope_time.sel(Nisf=nisf_of_int),Nisf=nisf_of_int))

In [ ]:
plt.figure()
plt.scatter(file_isf['isf_area_here'],GLflux_weighted_mean/slope_after.sel(time=max_slope_time))

In [ ]:
plt.figure()
plt.scatter(prop_to_plot['TF_mean'].sel(scenario='ssp585',time=max_slope_time),GLflux_weighted_mean/slope_after.sel(time=max_slope_time))

In [ ]:
plt.figure()
plt.scatter(prop_to_plot['TF_mean'].sel(scenario='ssp585',time=max_slope_time),GLflux_weighted_mean/slope_after.sel(time=max_slope_time))

In [ ]:
anom_TF = prop_to_plot['TF_mean'].sel(scenario='ssp585') - prop_to_plot['TF_mean'].sel(scenario='historical',time=range(1850,1901)).mean('time')

In [ ]:
plt.figure()
plt.scatter(anom_TF.sel(time=max_slope_time),GLflux_weighted_mean/slope_after.sel(time=max_slope_time)) #/GLflux_weighted_mean

In [ ]:
file_viability_info['MASS_BALANCE'].sel(time=range(1850,2101), Nisf=kisf, scenario=scen, param=mparam, m=mm)

In [ ]:
file_isf['isf_name'].sel(Nisf=48).load()

In [ ]:
mean_prop_weighted.mean(['model','param','m']).sel(scenario='ssp585')

In [ ]:
inputpath_tas = '/bettik/burgardc/DATA/SUMMER_PAPER/interim/CMIP_TEMP/'
mod_list = ['ACCESS-CM2','ACCESS-ESM1-5','CNRM-CM6-1','CNRM-ESM2-1','GISS_E2-1-H',
            'CESM2-WACCM','CESM2','CanESM5','IPSL-CM6A-LR','MRI-ESM2-0','MPI-ESM1-2-HR',
            'GFDL-CM4','GFDL-ESM4','UKESM1-0-LL']

tas_mod_list = []

for mod in mod_list:
    print(mod)
    if mod not in ['GISS_E2-1-H']:
        tas_scen_list = []
        for scen in ['historical','ssp126','ssp245','ssp585']:
            if (mod == 'GFDL-CM4') and (scen == 'ssp126'):
                tas_mod_scen['tas'] = tas_mod_scen['tas']*np.nan
            else:
                tas_mod_scen = xr.open_dataset(inputpath_tas+mod+'/tas_Amon_'+mod+'_'+scen+'_fldmean_ymean.nc')
                #tas_mod_scen = xr.open_dataset(inputpath_tas+mod+'/tas_Amon_'+mod+'_'+scen+'_ymean_fldmeansouthof60.nc')
                tas_mod_scen['time'] = tas_mod_scen.time.dt.year
            tas_out = tas_mod_scen['tas'].squeeze(drop=True)
            if 'height' in tas_out.coords:
                tas_out = tas_out.drop('height')
            tas_scen_list.append(tas_out.assign_coords({'scenario':scen}))

        tas_all_scen = xr.concat(tas_scen_list,dim='scenario')
        tas_mod_list.append(tas_all_scen.assign_coords({'model':mod}))
        
tas_all_mod = xr.concat(tas_mod_list, dim='model') 

In [ ]:
tas_out

In [ ]:
tas_all_mod_ano = tas_all_mod - tas_all_mod.sel(scenario='historical',time=range(1850,1901)).mean('time')
glob_T_ano_weighted = tas_all_mod_ano.sel(scenario='ssp585').sel(model=model_2300).weighted(bay_weights_2300 * sens_weights.sel(model=model_2300))

In [ ]:
mean_glob_T = glob_T_ano_weighted.mean(['model','param','m'])

In [ ]:
plt.figure()
plt.scatter(mean_glob_T.sel(time=max_slope_time),prop_to_plot['TF_mean'].sel(scenario='historical',time=range(1850,1900)).mean('time'))

In [ ]:
plt.figure()
plt.scatter(mean_glob_T.sel(time=max_slope_time),prop_to_plot['TF_mean'].sel(scenario='ssp585',time=max_slope_time))

ON THE MODEL SCALE (not very conclusive)

In [ ]:
BMB_weighted_mean_mod = file_viability_info['BMB'].sel(model=model_2300).weighted(bay_weights_2300 * sens_weights.sel(model=model_2300)).mean(['param','m'])


In [ ]:
smoothed_curve = BMB_weighted_mean_mod.rolling(time=60,center=True).mean('time')
slope_before = (smoothed_curve.sel(scenario='ssp585') - smoothed_curve.shift(time=30).sel(scenario='ssp585'))
slope_after = (smoothed_curve.shift(time=-30).sel(scenario='ssp585') - smoothed_curve.sel(scenario='ssp585'))
slope_diff = slope_after - slope_before
slope_diff2 = smoothed_curve.shift(time=-30).sel(scenario='ssp585') - smoothed_curve.shift(time=30).sel(scenario='ssp585')
max_slope_time = slope_diff.time.isel(time=slope_diff.argmax('time'))

In [ ]:
for mmod in model_2300:
    plt.figure()
    plt.scatter(tas_all_mod_ano.sel(scenario='ssp585',time=max_slope_time.sel(model=mmod),model=mmod),file_TS3['TF_mean'].sel(scenario='historical',time=range(1850,1900),model=mmod).mean('time'))

In [ ]:
plt.scatter(mean_glob_T.sel(time=max_slope_time),prop_to_plot['TF_mean'].sel(scenario='ssp585',time=max_slope_time))

In [ ]:
plt.figure()
plt.scatter(mean_glob_T.sel(time=max_slope_time),prop_to_plot['TF_mean'].sel(scenario='historical',time=range(1850,1900)).mean('time'))

In [ ]:
plt.scatter(mean_glob_T.sel(time=max_slope_time),BMB_weighted_mean.sel(scenario='ssp585',time=max_slope_time)/file_isf['isf_area_here'])